<a href="https://colab.research.google.com/github/natreis02/Initial_Optmization_Concepts/blob/main/Detec%C3%A7%C3%A3o_de_sarcasmo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [40]:
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [41]:
vocab_size = 10000 # Usar 1000 palavras mais frquentes e ignorar as outras.
embedding_dim = 16 # Dimensão do vetor de entrada.
max_length = 100  # Define os tamanhos das sequências.
trunc_type='post' # Pode cortar palavras no início ou no final.
padding_type='post' # Preenche sentenças pequenas com 0's. Todas as sentenças tem que ter tamanhos iguais.
oov_tok = "<OOV>" # Out Of Vocabulary - palavras que estão fora do vocabulário.
training_size = 20000 # Tamanho do conjunto de treino.

In [42]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json


--2024-10-22 14:07:35--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 142.250.141.207, 74.125.137.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.1s    

2024-10-22 14:07:36 (56.1 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [43]:
with open("/tmp/sarcasm.json", 'r') as f: # Coleta dados sobre manchetes de revistas. O arquivo json é aberto no modo de leitura r.
    datastore = json.load(f) # O conteúdo é carregado na variável datashore.

sentences = [] # vai armazenar as manchetes de notícias.
labels = [] # vai armazenar os rótulos, indicando se a manchete é sarcástica ou não.

for item in datastore: # um laço for percorre cada item (dicionário) no datastore.
    sentences.append(item['headline']) # Cada dicionário tem uma chave 'headline' (que contém a manchete) e uma chave 'is_sarcastic' (que contém um rótulo indicando se a manchete é sarcástica ou não).
    labels.append(item['is_sarcastic'])

In [44]:

training_sentences = sentences[0:training_size] # determina a quantidade de exemplos a serem usados para treinamento.
testing_sentences = sentences[training_size:] # determina a quantidade de exemplos a serem usados para o teste.
training_labels = labels[0:training_size] # a partir de quantos exemplos são usados no treino, define os rótulos correspondentes à cada sentença (0 ou 1).
testing_labels = labels[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok) # Utiliza as informações sobre cada variável definida anteriormente.
tokenizer.fit_on_texts(training_sentences) # Treina o tokenizer nas sentenças de treino. A variável 'training_sentences' percorre todas as sentenças e cria um vocabulário, associando um número único para cada palavra com base em sua frequência no conjunto de dados.

word_index = tokenizer.word_index # Está criando um dicionário (word_index) que mapeia cada palavra do vocabulário para um número inteiro, representando seu token correspondente.

training_sequences = tokenizer.texts_to_sequences(training_sentences) # Converte as sentenças de treino em sequências de números inteiros (tokens) e, em seguida, ajustando o comprimento dessas sequências para garantir que todas tenham o mesmo tamanho.
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [45]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded) # Essas variáveis contêm as sequências de entrada preenchidas/truncadas e convertidas para array para entrada do modelo.
training_labels = np.array(training_labels) # Essas variáveis contêm os rótulos correspondentes às sentenças, indicando a classificação de cada uma e são convertidas para array.
testing_padded = np.array(testing_padded) # Essas variáveis contêm as sequências de entrada preenchidas/truncadas e convertidas para array para entrada do modelo.
testing_labels = np.array(testing_labels) # Essas variáveis contêm os rótulos correspondentes às sentenças, indicando a classificação de cada uma e são convertidas para array.

In [46]:
# Modelo de rede neural que treinará e classificará os dados.
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'), # usada em camadas intermediárias. Transforma valores negativos em zeros.
    tf.keras.layers.Dense(1, activation='sigmoid') # usada geralmente na saída. Usada para classificação binária.
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ ?                           │     0 (unbuilt) │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [47]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 4s - 6ms/step - accuracy: 0.5795 - loss: 0.6726 - val_accuracy: 0.6758 - val_loss: 0.6195
Epoch 2/30
625/625 - 2s - 4ms/step - accuracy: 0.7413 - loss: 0.5266 - val_accuracy: 0.7617 - val_loss: 0.4790
Epoch 3/30
625/625 - 3s - 4ms/step - accuracy: 0.8285 - loss: 0.3985 - val_accuracy: 0.8123 - val_loss: 0.4077
Epoch 4/30
625/625 - 4s - 6ms/step - accuracy: 0.8533 - loss: 0.3452 - val_accuracy: 0.8472 - val_loss: 0.3677
Epoch 5/30
625/625 - 2s - 4ms/step - accuracy: 0.8638 - loss: 0.3186 - val_accuracy: 0.8454 - val_loss: 0.3589
Epoch 6/30
625/625 - 3s - 4ms/step - accuracy: 0.8828 - loss: 0.2880 - val_accuracy: 0.8343 - val_loss: 0.3714
Epoch 7/30
625/625 - 2s - 4ms/step - accuracy: 0.9011 - loss: 0.2511 - val_accuracy: 0.8481 - val_loss: 0.3558
Epoch 8/30
625/625 - 3s - 5ms/step - accuracy: 0.9017 - loss: 0.2407 - val_accuracy: 0.8083 - val_loss: 0.4421
Epoch 9/30
625/625 - 3s - 5ms/step - accuracy: 0.9057 - loss: 0.2300 - val_accuracy: 0.8243 - val_loss: 0.4048
E

In [48]:
# É um dicionário reverso que converte os tokens de volta as palavras.
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sentence(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text]) # Se o token não estiver no reverse_word_index, ele retorna '?'

print(decode_sentence(training_padded[0]))
print(training_sentences[2])
print(labels[2])

former <OOV> store clerk sues over secret 'black <OOV> for minority shoppers ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
mom starting to fear son's web series closest thing she will have to grandchild
1


In [49]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)


(10000, 16)


In [50]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8') # Este arquivo será usado para armazenar os vetores de embeddings. Cada linha representará um vetor associado a uma palavra.
out_m = io.open('meta.tsv', 'w', encoding='utf-8') # Este arquivo armazenará as palavras correspondentes a cada vetor de embedding. Cada linha conterá uma palavra.
for word_num in range(1, vocab_size): # O loop itera sobre os índices do vocabulário.
  word = reverse_word_index[word_num] # Em 'word', para cada número de palavra (word_num), ele busca a palavra correspondente no reverse_word_index.
  embeddings = weights[word_num]
  out_m.write(word + "\n") # Adiciona a palavra no arquivo meta.tsv, uma por linha.
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n") # Converte cada valor do vetor de embeddings em uma string e os junta com tabulações (\t). Essa linha é então escrita no arquivo vecs.tsv.
out_v.close() # Arquivos são fechado para garantir que todas as operações sejam garantidas.
out_m.close() # Arquivos são fechado para garantir que todas as operações sejam garantidas.

In [51]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [62]:
sentence = ["Oh great, another thrilling episode of waiting for my laundry to dry", "I clapped because it’s finished, not because I like it", "Oh, wonderful! Another Monday. Just what I needed to start my week!"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
[[0.99706715]
 [0.223831  ]
 [0.08308772]]
